In [ ]:
import os
import re
import boto3
import urllib3

import pandas as pd

from GOSTrocks.misc import tPrint

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
bucket = "wbg-geography01"
prefix = "URBANIZATION/MR_Novel_Poverty"
region = "us-east-1"
s3client = boto3.client("s3", region_name=region, verify=False)

# Loop through the S3 bucket and get all the file keys
more_results = True
try:
    del token  # noqa
except Exception:
    pass
loops = 0

all_res = []
while more_results:
    print(f"Completed loop: {loops}")
    if loops > 0:
        objects = s3client.list_objects_v2(
            Bucket=bucket,
            ContinuationToken=token,  # noqa
            Prefix=prefix,  # noqa
        )
    else:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    more_results = objects["IsTruncated"]
    if more_results:
        token = objects["NextContinuationToken"]
    loops += 1
    for res in objects["Contents"]:
        # Check if the key ends with .csv
        if res["Key"].endswith("_1k.txt"):
            all_res.append(res)

In [ ]:
def extract_numbers(text):
    res = re.findall(r"\d+", text)
    if len(res) == 0:
        return None
    else:
        return float(res[0])


country_population_res = []
for res in all_res:
    # Get text file from S3
    res_text = s3client.get_object(Bucket=bucket, Key=res["Key"])["Body"].read()
    cntry = os.path.basename(res["Key"]).split("_")[-2]
    tPrint(f"Processing {cntry}...")
    # Go through lines of results and extract urbanization numbers
    res_lines = []
    for line in res_text.decode("utf-8").split("\n"):
        if line == "***** Evaluate Urbanization *****":
            record_results = True
        elif line == "***** Evaluate SMOD ******":
            record_results = False
        if record_results:
            res_lines.append(line)

    # Extract the summary results for all population layers
    res_lines = res_lines[1:]  # Remove the first line which is a header
    all_pop_res = []
    for line in res_lines:
        line_split = line.split(":")
        pop_layer = line_split[0].strip()
        pop_res = [extract_numbers(x) for x in line_split[1:]]
        pop_res.append(pop_layer)
        if len(pop_res) >= 4:
            all_pop_res.append(pop_res[1:])
    country_res = pd.DataFrame(
        all_pop_res, columns=["Population", "HD_Urban", "Urban", "PopLayer"]
    )
    country_res["Country"] = cntry
    country_population_res.append(country_res)

In [ ]:
pd.concat(country_population_res).to_csv(
    "C:/WBG/Work/temp/urbanization_country_population.csv", index=False
)

In [ ]:
res_lines[1].split(":")

In [ ]:
res_lines

In [ ]:
_l